In [7]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
#sensor data
#Imu sensor data
a_k_1 = np.identity(3)

#GPS data
host_pos = np.array((1,2))

In [32]:
#Prediction model function


def Prediction(p_est, v_est, q_est, dt, a_k_1, w_k_1):
    '''
    Input:
    p_est : estimated position array from previous cycle
    v_est : estimated velocity array from previous cycle
    q_est : estimated quaternion array from previous cycle
    dt    : delta t
    
    Returns:
    p_pred : predicted position for current cycle
    v_pred : predicted velocity for current cycle
    q_est  : predicted quaternion for current cycle
    '''
    g = np.array([0,0,-9.81])
    R, quaternion_k_1 = Quaternion(w_k_1)
    
    p_pred = p_est + v_est * dt + np.dot(R,a_k_1 + g)*dt**2/2
    v_pred = v_est + np.dot(R,a_k_1 + g)*dt
    
    _, current_rotation_quaternion = Quaternion(w_k_1 * dt)
    q_pred = Quaternion_multiplication(current_rotation_quaternion, quaternion_k_1)

In [34]:
# Update linearized prediction model Jacobians
def update_MM_jacobians(dt, R, acc_k_1):
    '''
    Input:
    dt      : delta t
    R       : a 3*3 rotational matrix
    acc_k_1 : a 3*1 array of acceleration values
    
    Output:
    F : a 9*9 state transition matrix
    Q : a 6*6 process noise covariance matrix
    '''
    F = np.identity(9)
    Q = np.identity(6)
    
    F[:3, 3:6] = np.identity(3) * dt
    F[3:6,6:] = -np.dot(R, skew_symmetric(acc_k_1))
    
    Q[:, :3] *= var_imu_acc * dt**2
    Q[:,-3:] *= var_imu_w * dt**2

In [ ]:
# propagate uncertainity
def propogate_uncertainity(F,Q, L_jac, p_cov_est):
    '''
    Input:
    F     : a 9*9 state transition matrix
    Q     : a 6*6 process noise covariance matrix
    L_jac : motion model noise jacobian matrix
    p_cov_est : a 9*9 estimates covariance matrix from last cycle
    
    Output:
    p_cov_pred : a 9*9 predicted covariance matrix for current cycles
    '''
    
    p_cov_pred = np.dot(np.dot(F,p_cov_est),F.T) + np.dot(np.dot(L_jac,Q),L_jac.T)
    
    return p_cov_pred   
    

In [7]:
# measurement update function

In [4]:
#normalize angle function
def wrap_to_pi(angle):
    '''
    Input:
    angle : an angle in radians
    
    Output : normalized angle in radians
    '''
    
    if angle > np.pi:
        angle -= 2*np.pi
    elif angle < np.pi:
        angle += 2*np.pi
    
    return angle

In [5]:
#Quaternion function
def Quaternion(arg, rot):
    '''
    Input:
    arg : calculate rotational matrix or perform quaternion multiplication
    rot : a 3*1 np.ndarray with rotation information wrt 3 axes
    
    Output:
    R : a 3*3 np.ndarray (rotational matrix)
    '''
    
    roll = wrap_to_pi(rot[0])
    pitch = wrap_to_pi(rot[1])
    yaw = wrap_to_pi(rot[2])
    
    cos_roll = np.cos(roll/2)
    sin_roll = np.sin(roll/2)
    cos_pitch = np.cos(pitch/2)
    sin_pitch = np.sin(pitch/2)
    cos_yaw = np.cos(yaw/2)
    sin_yaw = np.sin(yaw/2)
    
    # real component of quaternion
    w = cos_roll * cos_pitch * cos_yaw + sin_roll + sin_pitch + sin_yaw
    
    # imaginary component of quaternion
    x = sin_roll * cos_pitch * cos_yaw + sin_roll * sin_pitch * sin_yaw
    y = cos_roll * sin_pitch * cos_yaw + sin_roll * cos_pitch * sin_yaw
    z = cos_roll * cos_pitch * sin_yaw - sin_roll * sin_pitch * cos_yaw
    
    quaternion = np.array([w,x,y,z]).reshape(4,1)
    
    vec = np.array([x,y,z]).reshape(3,1)
    
    Skew_symmetric_vec = np.array([[0, -z, y],
                                  [z, 0 ,x],
                                  [-y, x, 0]])
    
    R = ((w**2 - np.dot(vec.T,v)) * np.identity(3)) + (2 * np.dot(v,v.T)) + (2 * w * Skew_symmetric_vec)
    
    return R, quaternion
    
    

In [ ]:
#Quaternion multiplication right
quaternion =  Quaternion_multiplication(q1, q2):
    '''
    input : 
    q1. q2 : 4*1 array of quaternion components [w,x,y,z]
    
    output :
    quaternion = multiplication result of q1 and q2
    '''
    w = q1[0]
    x = q1[1]
    y = q1[2]
    z = q1[3]
    
    vec = np.array([x,y,z]).reshape(3,1)
    Skew_symmetric_vec = np.array([[0, -z, y],
                                  [z, 0 ,x],
                                  [-y, x, 0]])
    
    term1 = np.zeros([4,4])
    term1[0,1] = -v[:,0]
    term1[1:,0] = v[:,0]
    term1[1:,1:] = -Skew_symmetric_vec
    
    term2 = w * np.identity(4) + term1
    
    quaternion = np.dot(term2, q2)
        
    return quaternion    